This notebook is modified from the unbond-generics example on hackage and it repo.

If you have not run `./install.sh` alreay on Jupyter terminal, and running it here for the first time,
`./install.sh` will install dependencies specified in the script into the LTS snapshot directory.
However, on the very first run, it will fail to import modules becuase the running kernel would still be oblivious to the newly installed pakcages.
In such case, you should restart the kernel. Once the kernel is restarted, the `./install.sh` won't do anyting because everything is to date and the newly loaded kernel should be able to see the newly installed package modules.

In [1]:
:!./install.sh

In [2]:
{-# LANGUAGE DeriveDataTypeable
           , DeriveGeneric
           , FlexibleInstances
           , FlexibleContexts
           , MultiParamTypeClasses #-}

import Unbound.Generics.LocallyNameless
import Unbound.Generics.LocallyNameless.Internal.Fold

import GHC.Generics (Generic)
import Data.Typeable (Typeable)

In [3]:
-- | Names for expressions
type Nm = Name Exp


-- | Expressions
data Exp = Var Nm            -- ^ variables
         | Lam (Bind Nm Exp) -- ^ lambdas bind a variable within a body expression
         | App Exp Exp       -- ^ application
          deriving (Show, Generic, Typeable)

-- Automatically construct alpha equiv, free vars, and binding operations.
instance Alpha Exp

-- semi-automatically implement capture avoiding substitution
instance Subst Exp Exp where
  -- `isvar` identifies the variable case in your AST.
  isvar (Var x) = Just (SubstName x)
  isvar _       = Nothing

In [4]:
import Unbound.Generics.LocallyNameless.Unsafe
:type unsafeUnbind

unsafeUnbind :: forall p t. (Alpha p, Alpha t) => Bind p t -> (p, t)

In [5]:
import IHaskell.Display

showExp = ppExp "λ" "."
printExp = putStrLn . showExp
dispExp e = html $ "<code>"++ppExp "λ" "." e++"</code>"
displayExp = display . dispExp

ppExp _ _ (Var x) = show x
ppExp l d (Lam bnd) = l ++ show x ++ d ++ ppExp l d t
  where (x, t) = unsafeUnbind bnd
ppExp l d (App t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam{}) = paren (ppExp l d t)
  ppt t         = ppExp l d t
  pps s@(Var{}) = ppExp l d s
  pps s         = paren (ppExp l d s)

paren s = "("++s++")"

In [6]:
:type printExp
:type dispExp
:type displayExp

printExp :: Exp -> IO ()

dispExp :: Exp -> DisplayData

displayExp :: Exp -> IO Display

In [7]:
-- call-by-value evaluation
-- evaluation takes an expression and returns a value while using a source of fresh names
eval (Var x)     = fail $ "unbound variable " ++ show x
eval e@(Lam _)  = return e
eval (App e1 e2) = do
  v1 <- eval e1
  v2 <- eval e2
  case v1 of
   (Lam bnd) -> do
     -- open the lambda by picking a fresh name for the bound variable x in body
     (x, body) <- unbind bnd
     eval (subst x v2 body)
   _ -> fail "application of non-lambda"
   
:type eval

eval :: forall (m :: * -> *). Fresh m => Exp -> m Exp

In [8]:
x = s2n "x"
y = s2n "y"
z = s2n "z"
e = Lam $ bind x (Lam $ bind y (App (Var y) (Var x)))

In [9]:
lam x = Lam . bind x
:type lam

lam :: Nm -> Exp -> Exp

In [10]:
lam x (Var x)
lam y (Var y)

dispExp $ lam x (Var x)
dispExp $ lam y (Var y)

lam x (Var x) `aeq` lam y (Var y)

Lam (<x> Var 0@0)

Lam (<y> Var 0@0)

λx.x

λy.y

True

In [11]:
e
dispExp e

Lam (<x> Lam (<y> App (Var 0@0) (Var 1@0)))

λx.λy.y x

In [12]:
dispExp (App (App e e) e)
dispExp $ runFreshM $ eval (App (App e e) e)

(λx.λy.y x) (λx.λy.y x) (λx.λy.y x)

λy.y (λx.λy.y x)

In [13]:
tmId = lam x (Var x)
tm1 = App tmId tmId `App` App tmId tmId
tm2 = lam y $ App tmId tmId `App` App tmId tmId

In [14]:
dispExp tmId
dispExp $ runFreshM $ eval tmId
dispExp tm1
dispExp $ runFreshM $ eval tm1
dispExp tm2
dispExp $ runFreshM $ eval tm2

λx.x

λx.x

(λx.x) (λx.x) ((λx.x) (λx.x))

λx.x

λy.(λx.x) (λx.x) ((λx.x) (λx.x))

λy.(λx.x) (λx.x) ((λx.x) (λx.x))

In [15]:
:type fv
:type toListOf
fv' e = toListOf fv e :: [Nm]
:type fv'

fv :: forall a b (f :: * -> *). (Alpha a, Typeable b, Contravariant f, Applicative f) => (Name b -> f (Name b)) -> a -> f a

toListOf :: forall s a. Fold s a -> s -> [a]

fv' :: forall s. Alpha s => s -> [Nm]

In [16]:
fv' tmId
fv' (Var x)
fv' (Var x `App` Var y)
fv' (lam x . lam y $ (Var x `App` Var y) `App` Var z)

[]

[x]

[x,y]

[z]

In [17]:
-- reduce all the way (may diverge)
red (App e1 e2) = do
  e1' <- red e1
  e2' <- red e2
  case e1' of
    Lam bnd -> do
        (x, e1'') <- unbind bnd
        return $ subst x e2' e1''
    _ -> return $ App e1' e2'
red (Lam bnd) = do
   (x, e) <- unbind bnd
   e' <- red e
   case e of
     App e1 (Var y) | y == x && x `notElem` fv' e1 -> return e1
     _ -> return $ lam x e'
red (Var x) = return $ Var x

:type red

red :: forall (m :: * -> *). Fresh m => Exp -> m Exp

In [18]:
tm2
dispExp $ runFreshM $ eval tm2
dispExp $ runFreshM $ red tm2

Lam (<y> App (App (Lam (<x> Var 0@0)) (Lam (<x> Var 0@0))) (App (Lam (<x> Var 0@0)) (Lam (<x> Var 0@0))))

λy.(λx.x) (λx.x) ((λx.x) (λx.x))

λy.λx5.x5

In [19]:
import Control.Applicative 

-- single step reduction
step e@(App e1 e2) = beta e
                 <|> (App <$> step e1 <*> pure e2)
                 <|> (App <$> pure e1 <*> step e2)
step (Lam bnd) = do (x, e) <- unbind bnd
                    lam x <$> step e
step (Var x) = empty


beta (App (Lam bnd) e2) = do (x, e1) <- unbind bnd
                             return $ subst x e2 e1
beta _                  = empty


:type step
:type beta

step :: forall (f :: * -> *). (Alternative f, Fresh f) => Exp -> f Exp

beta :: forall (m :: * -> *). (Fresh m, Alternative m) => Exp -> m Exp

In [20]:
omega = lam x $ Var x `App` Var x 
tmOmega = App omega omega -- well known diverging term

dispExp omega
dispExp tmOmega

λx.x x

(λx.x x) (λx.x x)

In [21]:
:type runFreshMT . step $ tmOmega
map dispExp . runFreshMT . step $ tmOmega

runFreshMT . step $ tmOmega :: forall (m :: * -> *). MonadPlus m => m Exp

(λx.x x) (λx.x x)

In [22]:
:type runFreshMT . step $ tm1
map dispExp . runFreshMT . step $ tm1

runFreshMT . step $ tm1 :: forall (m :: * -> *). MonadPlus m => m Exp

(λx.x) ((λx.x) (λx.x))

(λx.x) (λx.x) (λx.x)

In [23]:
:type runFreshMT . step $ tm2
map dispExp . runFreshMT . step $ tm2

runFreshMT . step $ tm2 :: forall (m :: * -> *). MonadPlus m => m Exp

λy.(λx.x) ((λx.x) (λx.x))

λy.(λx.x) (λx.x) (λx.x)

In [29]:
tm10 = (App (lam x $ Var x) (lam y $ Var y) `App` App (lam x $ Var x) (lam z $ Var z))

list1 :: [Exp]
list1 = runFreshMT . step $ tm10
list2 :: [[Exp]]
list2 = map (runFreshMT . step) list1

dispExp tm10
putStrLn "list1:"
map dispExp list1
putStrLn "list2:"
map (map dispExp) list2

(λx.x) (λy.y) ((λx.x) (λz.z))

list1:

(λy.y) ((λx.x) (λz.z))

(λx.x) (λy.y) (λz.z)

list2:

(λx.x) (λz.z)

(λy.y) (λz.z)

(λy.y) (λz.z)

In [25]:
import Data.Tree hiding (drawTree)
import Data.Tree.View

redTree e = Node e (map redTree . runFreshMT . step $ e)

In [26]:
:type showTree . fmap showExp
drawTreeExp = showTree . fmap showExp

showTree . fmap showExp :: Tree Exp -> String

In [27]:
putStrLn $ drawTreeExp $ redTree (App (lam x $ Var x) (lam y $ Var y) `App` App (lam x $ Var x) (lam z $ Var z))

(λx.x) (λy.y) ((λx.x) (λz.z))
 ├╴(λy.y) ((λx.x) (λz.z))
 │  ├╴(λx.x) (λz.z)
 │  │  └╴λz.z
 │  └╴(λy.y) (λz.z)
 │     └╴λz.z
 └╴(λx.x) (λy.y) (λz.z)
    └╴(λy.y) (λz.z)
       └╴λz.z